In [20]:
import cv2
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from collections.abc import Iterable
from tensorflow.keras.utils import normalize

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

In [9]:
input_size = 64
image_directory = 'datasets/'

no_tumor_images = os.listdir(image_directory + 'no/')

yes_tumor_images = os.listdir(image_directory+'yes/')

dataset=[]

label =[]


In [10]:
for i,image_name in enumerate(no_tumor_images):
    if image_name.lower().endswith(('.jpg', '.jpeg')):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image,'RGB')
        image = image.resize((input_size,input_size))
        dataset.append(np.array(image))
        label.append(0)

for i,image_name in enumerate(yes_tumor_images):
    if image_name.lower().endswith(('.jpg', '.jpeg')):
        image=cv2.imread(image_directory+'yes/'+image_name)
        image=Image.fromarray(image,'RGB')
        image = image.resize((input_size,input_size))
        dataset.append(np.array(image))
        label.append(1)


In [11]:
dataset = np.array(dataset)
label = np.array(label)

In [21]:
x_train,x_test,Y_train,y_test = train_test_split(dataset,label,test_size=0.2,random_state=0)


#reshape = (n, image_width, image_height, n_channel)


# print(x_train.shape)
# print(x_test.shape)

x_train = normalize(x_train,axis=1)
x_test = normalize(x_test,axis=1)


Y_train=to_categorical(Y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)

In [26]:
#Model_building 

model = Sequential()

model.add(Conv2D(32,(3,3),input_shape =(input_size,input_size,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

# Binary CrossEntropy=1,sigmoid
# categorical Cross Entropy = 2,softmax

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x_train,Y_train,batch_size=16,verbose = 1, epochs=10, validation_data= (x_test,y_test),shuffle=False)

model.save("Brain_Tumor10EpochsCategorical.h5")

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.5780 - loss: 0.6783 - val_accuracy: 0.6863 - val_loss: 0.5900
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6996 - loss: 0.5624 - val_accuracy: 0.8039 - val_loss: 0.4798
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7448 - loss: 0.5916 - val_accuracy: 0.8039 - val_loss: 0.4622
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7084 - loss: 0.5612 - val_accuracy: 0.8235 - val_loss: 0.4262
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7762 - loss: 0.4897 - val_accuracy: 0.8627 - val_loss: 0.4170
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7742 - loss: 0.4623 - val_accuracy: 0.8824 - val_loss: 0.4106
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7886 - loss: 0.4345 - val_accuracy: 0.8824 - val_loss: 0.3962
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8478 - loss: 0.3789 - val_accuracy: 0.8627 - v

In [24]:
model.evaluate

<bound method TensorFlowTrainer.evaluate of <Sequential name=sequential_7, built=True>>

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load the ResNet50 model with pre-trained ImageNet weights (without the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50 (optional if you want to fine-tune)
base_model.trainable = False

# Build the full model
model = Sequential()

# Add the base ResNet50 model
model.add(base_model)

# Flatten the output of the base model
model.add(Flatten())

# Add a dense layer with 64 units
model.add(Dense(64, activation='relu'))

# Add dropout layer to avoid overfitting
model.add(Dropout(0.5))

# Add the output layer (2 units for binary classification, softmax activation for multi-class)
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Assuming you have preprocessed images ready, like x_train, y_train, x_test, y_test
# If using a data generator, you can preprocess the images as follows:
# Example: using ImageDataGenerator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Replace with your own directories
train_generator = train_datagen.flow_from_directory(
    x_train,  # Path to training images
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'  # Change to 'categorical' for multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    x_test,  # Path to testing images
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'  # Change to 'categorical' for multi-class classification
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=2000 // 16,  # Replace with your dataset size
    epochs=10,
    validation_data=test_generator,
    validation_steps=800 // 16  # Replace with your dataset size
)

# Save the trained model
model.save("Brain_Tumor_ResNet50_Categorical.h5")

# Evaluate the model on test data
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 199s 2us/step


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │     6,422,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,010,434 (114.48 MB)

 Trainable params: 6,422,722 (24.50 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

ValueError: listdir: embedded null character in path